In [2]:
from llama_cpp import Llama

In [3]:
llm = Llama(model_path="/data/mistral/query-to-mql/exp-10/nov-20/merged-model/ggml-model-q4_0/ggml-model-q4_0.gguf")

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [6]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

context = """{
    "MEASURE": [{"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]}],
    "FILTER": [{"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [7]:
user_query = 'sales in nov 2020 to may 2021 vs Q3 2021'

inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)

In [ ]:
output = llm(
  inp, # Prompt
  max_tokens=200, # Generate up to 32 tokens
  echo=True # Echo the prompt back in the output
)

In [ ]:
print(output)